In [1]:
import requests

In [23]:
headers = { "host": "sample-240.h5files.hdfserver"}
hallo = requests.get("http://hdfserver:5000/update", headers=headers)

In [22]:
hallo

<Response [404]>

In [21]:
hallo.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)